In [12]:
# See: https://github.com/terminalmage/adventofcode/blob/main/2022/day17.py
import itertools

Coordinate = tuple[int, int]
Rock = set[Coordinate]

class AoC_Day17():
    def __init__(self, example: bool) -> None:
        '''
        Set up jet pattern and rock sequence. 
        '''

        # super().__init__(example=example)
        if example:
            s = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
        else:
            with open("aoc_17_input.txt", "r") as f:
                s = f.read().strip()
        
        jet_map = {'<' : 'left', '>': 'right'}
        self.__jet_pattern = tuple(
            jet_map[item] for item in s
        )
        
        self.width = 7
        self.chamber = set()
        self.top = 0
        self.__rock_sequence = (
            # @@@@
            lambda : {
                (2, self.top + 4), (3, self.top + 4), (4, self.top + 4), (5, self.top + 4)
            },
            # + 
            lambda : {
                                    (3, self.top + 6), 
                (2, self.top + 5),  (3, self.top + 5), (4, self.top + 5), 
                                    (3, self.top + 4)
            }, 
            # corner
            lambda : {
                                                       (4, self.top + 6),
                                                       (4, self.top + 5),
                (2, self.top + 4),  (3, self.top + 4), (4, self.top + 4), 
            }, 
            # | 
            lambda : {
                (2, self.top + 7), (2, self.top + 6), (2, self.top + 5), (2, self.top + 4)
            },
            # box
            lambda : {
                (2, self.top + 5), (3, self.top + 5), 
                (2, self.top + 4), (3, self.top + 4)
            }
        )

    def move_down(self, rock: Rock) -> Rock:
        next_rock = {(col, row - 1) for col, row in rock}
        return rock if next_rock & self.chamber else next_rock

    def move_left(self, rock: Rock) -> Rock: 
        next_rock = {(col-1, row) for col, row in rock}
        if any(coord[0] < 0 for coord in next_rock) or next_rock & self.chamber:
            return rock
        return next_rock
    
    def move_right(self, rock: Rock) -> Rock: 
        next_rock = {(col+1, row) for col, row in rock}
        if any(coord[0] >= self.width for coord in next_rock) or next_rock & self.chamber:
            return rock
        return next_rock
    
    def reset_chamber(self) -> None:
        self.chamber.clear()
        self.chamber.update((col,0) for col in range(self.width))
        self.top = 0
        self.jet_pattern = itertools.cycle(enumerate(self.__jet_pattern))
        self.rock_sequence = itertools.cycle(enumerate(self.__rock_sequence))

    def print_chamber(self) -> None:
        for row in range(self.top, 0, -1):
            print(f"|{''.join(['#'  if (col, row) in self.chamber  else '.' for col in range(self.width)])}|")
        print(f"+{''.join(['-' for _ in range(self.width)])}+")

    def part1(self) -> int:
        return self.tetris(2022)

    def part2(self) -> int:
        return self.tetris(1_000_000_000_000)

    def tetris(self, num_rocks : int) -> int:
    # calculate chamber height after #num_rocks rocks
        self.reset_chamber()
        for rock_num in range(num_rocks):
            #if (rock_num % 200) == 0:
            #    print()
            #    self.print_chamber()
            rock_index, rock_gen = next(self.rock_sequence)
            rock = rock_gen()
            while True:
                jet_index, direction = next(self.jet_pattern)
                # > or <
                rock = getattr(self, f'move_{direction}')(rock)
                # v
                new_pos = self.move_down(rock)
                if new_pos == rock:
                    # no more downward movement
                    self.chamber.update(rock)
                    self.top = max(coord[1] for coord in self.chamber)
                    break
                rock = new_pos
        return self.top



if __name__ == '__main__':
    aoc = AoC_Day17(example=False)
    print(f"Part 1: {aoc.part1()}")

        
    


Part 1: 3232


In [ ]:
# After looking up a solution for part 1, I want to put this down and come back to it when I feel confident I can do part 2 myself. 